In [1]:
import os
# Changing the working directory
os.chdir('../..')
os.getcwd()

'/home/javiermunoz/Universidad/MasterBDyDS/TFM/TFM_DSBD'

In [2]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

import tensorflow as tf
gpu_devices = tf.config.experimental.list_physical_devices("GPU")
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc("figure", figsize=(15, 5))

import numpy as np
import pandas as pd

/home/javiermunoz/anaconda3/envs/tf/lib/python3.9/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (None)/charset_normalizer (3.1.0) doesn't match a supported version!
  warnings.warn(


In [3]:
leaderboard = pd.read_csv("data/leaderboard.csv")
leaderboard

,model_type,model_name,train_acc,val_acc,test_acc,train_loss,val_loss,test_loss,epoch,path_to_model,subset_size
0,convlstm,simple,0.96,0.93,0.93,0.15,0.20,0.19,39.0,src/convlstm/bestmodels/best_simple_39_0.93_10...,10.0
1,convlstm,stateless,0.93,0.95,0.91,0.22,0.19,0.19,22.0,src/convlstm/bestmodels/best_stateless_22_0.95...,10.0
2,mediapipe,mp,0.90,0.90,0.87,0.27,0.35,0.54,178.0,src/mediapipe/bestmodels/best_mp_178_0.90_20_m...,20.0
3,mediapipe,mp,0.89,0.85,0.85,0.38,0.62,0.43,159.0,src/mediapipe/bestmodels/best_mp_159_0.85_40_m...,40.0
4,mediapipe,mp,0.93,0.86,0.86,0.24,0.63,0.55,167.0,src/mediapipe/bestmodels/best_mp_167_0.86_30_m...,30.0
5,3DCNN,3dcnn,0.93,0.92,0.91,0.33,0.37,0.40,132.0,src/3DCNN/bestmodels/best_3dcnn_132_0.92_20_20...,20.0
6,convlstm,stateless,0.90,0.94,0.92,0.27,0.24,0.22,63.0,src/convlstm/bestmodels/best_stateless_63_0.94...,20.0
7,convlstm,stateless,0.95,0.92,0.82,0.20,0.26,0.46,93.0,src/convlstm/bestmodels/best_stateless_91_0.92...,30.0
8,convlstm,stateless,0.87,0.77,0.74,0.38,1.05,1.06,104.0,src/convlstm/bestmodels/best_stateless_104_0.7...,40.0
9,convlstm,simple,0.83,0.65,0.65,0.48,1.36,1.27,170.0,src/convlstm/bestmodels/best_simple_170_0.65_4...,40.0


In [4]:
path_to_simple = leaderboard['path_to_model'][1]
path_to_stateless = leaderboard['path_to_model'][2]

model_to_test = tf.keras.models.load_model(path_to_stateless)
model_to_test.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 10, 128)           916992    
                                                                 
 dropout (Dropout)           (None, 10, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 10, 256)           394240    
                                                                 
 dropout_1 (Dropout)         (None, 10, 256)           0         
                                                                 
 lstm_2 (LSTM)               (None, 10, 128)           197120    
                                                                 
 dropout_2 (Dropout)         (None, 10, 128)           0         
                                                                 
 lstm_3 (LSTM)               (None, 64)                4

In [5]:
from src.utils import load_dict
subset = 10
labels=load_dict(f'data/subset_{subset}_lsa_64/pickl_files/labels_map.pkl')
labels

{'Barbecue': 0,
 'Birthday': 1,
 'Buy': 2,
 'Chewing-gum': 3,
 'Coin': 4,
 'Milk': 5,
 'Mock': 6,
 'Realize': 7,
 'Sweet milk': 8,
 'To land': 9}

## Inference based on test videos

In [6]:
import cv2
from src.load_dataset_aux import format_frames
from numpy.random import default_rng

rng = default_rng()
threshold = 0.7
output_size = (224,224)

cap = cv2.VideoCapture('/home/javiermunoz/TFM_DSBD/data/subset_10_lsa_64/test/Mock/029_001_004.mp4')

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

captured_frames=[]
count=1 
ret=True
while ret:
    ret, frame = cap.read()
    captured_frames.append(frame)

cap.release()
cv2.destroyAllWindows()
    
frames2extract_idx = np.array(sorted(rng.choice(len(captured_frames), size=10, replace=False)))

frames_to_predict = []
for i in range(len(captured_frames)):
    if i in frames2extract_idx:
        frames_to_predict.append(captured_frames[i])

result = []
for frame in frames_to_predict:
    frame_formatted = format_frames(frame, output_size)
    result.append(frame_formatted)

result = np.array(result)[..., [2,1,0]]

logits = model_to_test.predict(np.expand_dims(result, axis=0))
if logits[logits > threshold].any():
    pred = np.argmax(logits)
    pred_label = [k for k, v in labels.items() if v == pred]
    print(pred_label)

captured_frames = []



(python:1375): GStreamer-CRITICAL **: 11:28:53.740: gst_element_make_from_uri: assertion 'gst_uri_is_valid (uri)' failed
[ WARN:0@0.019] global /croot/opencv-suite_1676452025216/work/modules/videoio/src/cap_gstreamer.cpp (1127) open OpenCV | GStreamer warning: Error opening bin: no source element for URI "/home/javiermunoz/TFM_DSBD/data/subset_10_lsa_64/test/Mock/029_001_004.mp4"
[ WARN:0@0.019] global /croot/opencv-suite_1676452025216/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


ValueError: Cannot take a larger sample than population when replace is False

## Inference based on video camera

In [6]:
import cv2
from src.load_dataset_aux import format_frames
from numpy.random import default_rng

rng = default_rng()
threshold = 0.7
output_size = (224,224)

cap = cv2.VideoCapture(0)

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

captured_frames=[]
count=1 
ret=True
while ret:
    ret, frame = cap.read()
    captured_frames.append(frame)
    if count % 100 == 0:
        frames2extract_idx = np.array(sorted(rng.choice(len(captured_frames), size=10, replace=False)))
        
        frames_to_predict = []
        for i in range(len(captured_frames)):
            if i in frames2extract_idx:
                frames_to_predict.append(captured_frames[i])


        result = []
        for frame in frames_to_predict:
            frame_formatted = format_frames(frame, output_size)
            result.append(frame_formatted)
        
        result = np.array(result)[..., [2,1,0]]

        logits = model_to_test.predict(np.expand_dims(result, axis=0))
        print(logits)
        if logits[logits > threshold].any():
            pred = np.argmax(logits)
            pred_label = [k for k, v in labels.items() if v == pred]
            print(pred_label)

        captured_frames = []

    count += 1
    # Show to screen
    cv2.imshow('OpenCV Feed', frame)

    # Break gracefully
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


[ WARN:0@0.015] global /croot/opencv-suite_1676452025216/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
[ WARN:0@0.016] global /croot/opencv-suite_1676452025216/work/modules/videoio/src/cap_v4l.cpp (902) open VIDEOIO(V4L2:/dev/video0): can't open camera by index


error: OpenCV(4.6.0) /croot/opencv-suite_1676452025216/work/modules/highgui/src/window.cpp:967: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'imshow'
